# Kaggle 따라하기

# 1. 라이브러리 불러오기

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'missingno'

In [ ]:
train_data_path = join('./data', 'train.csv')
sub_data_path = join('./data', 'test.csv')

## 1. 데이터 살펴보기
pandas의 read_csv 함수를 사용해 데이터를 읽어오고, 각 변수들이 나타내는 의미를 살펴보겠습니다.
1. ID : 집을 구분하는 번호
2. date : 집을 구매한 날짜
3. price : 타겟 변수인 집의 가격
4. bedrooms : 침실의 수
5. bathrooms : 침실당 화장실 개수
6. sqft_living : 주거 공간의 평방 피트
7. sqft_lot : 부지의 평방 피트
8. floors : 집의 층 수
9. waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
10. view : 집이 얼마나 좋아 보이는지의 정도
11. condition : 집의 전반적인 상태
12. grade : King County grading 시스템 기준으로 매긴 집의 등급
13. sqft_above : 지하실을 제외한 평방 피트
14. sqft_basement : 지하실의 평방 피트
15. yr_built : 집을 지은 년도
16. yr_renovated : 집을 재건축한 년도
17. zipcode : 우편번호
18. lat : 위도
19. long : 경도
20. sqft_living15 : 2015년 기준 주거 공간의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)
21. sqft_lot15 : 2015년 기준 부지의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)

In [ ]:
#데이터 크기 확인
train = pd.read_csv(train_data_path)
test = pd.read_csv(sub_data_path)
print('train data dim : {}'.format(train.shape))
print('test data dim : {}'.format(test.shape))

In [ ]:
#타겟 변수 제거
y = train['price']
del train['price']

print(train.columns)

## 2. 간단한 전처리 
각 변수들에 대해 결측 유무를 확인하고, 분포를 확인해보면서 간단하게 전처리를 하겠습니다.

In [ ]:
#date 형 변환 -> str에서 int
train['date'] = train['date'].apply(lambda i: i[:6]).astype(int)
test['date'] = test['date'].apply(lambda i: i[:6]).astype(int)

train.head()

In [ ]:
#id 변수는 집값 예측에 도움이 되지 않아 삭제
del train['id']
del test['id']

In [ ]:
print(train.columns)

In [ ]:
#변수의 데이터 확인
fig, ax = plt.subplots(9, 2, figsize=(12, 50))   # 가로스크롤 때문에 그래프 확인이 불편하다면 figsize의 x값을 조절해 보세요. 

count = 0
columns = train.columns
for row in range(9):
    for col in range(2):
        sns.kdeplot(data=train[columns[count]], ax=ax[row][col])
        ax[row][col].set_title(columns[count], fontsize=15)
        count += 1
        if count == 19 :
            break

In [ ]:
#정규화가 필요한 변수에 대한 로그 변환
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']

for c in skew_columns:
    train[c] = np.log1p(train[c].values)
    test[c] = np.log1p(test[c].values)
    

In [ ]:
#라벨값의 분포 확인
sns.kdeplot(y)
plt.show()

In [ ]:
#라벨값 정규화
y = np.log1p(y)
y

In [ ]:
#정규화 결과 확인
sns.kdeplot(y)
plt.show()

In [ ]:
#int float으로만 구성되었는지 확인
train.info()

## 모델링에 필요한 라이브러리 불러오기

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from sklearn.model_selection import GridSearchCV

## 예측을 위한 모델링

In [ ]:
#RMSE 계산 함수
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

In [ ]:
random_state=919  # 고정값은 임의로 설정

#각 모델 변수 설정 후 models에 지정
gboost = GradientBoostingRegressor(random_state=random_state)
xgboost = XGBRegressor(random_state=random_state)
lightgbm = LGBMRegressor(random_state=random_state)
rdforest = RandomForestRegressor(random_state=random_state)

models = [gboost, xgboost, lightgbm, rdforest]

In [ ]:
#모델 성능 계산 함수
def get_scores(models, train, y):
    df = {}
    X_train, X_test, y_train, y_test = train_test_split(train, y, random_state=random_state, test_size=0.2)
    for model in models:
        model_name = model.__class__.__name__
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        df[model_name] = rmse(y_test, y_pred)
    return pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)
    
get_scores(models, train, y)

In [ ]:
#데이터 크기 확인
print(train.shape, y.shape)
temp_train = pd.concat([train, y], axis=1)
print(temp_train.shape)
temp_train.head()

In [ ]:
def my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5):
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', \
                              cv=2, verbose=verbose, n_jobs=n_jobs)
    
    # 모델 fitting
    grid_model.fit(train, y)

    # 결과값 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임 생성
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE 값 계산 후 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')
    
    return results

경고문에서 패키지 오류가 났는데 pip install -U setuptools 명령어를 통해 해결

In [ ]:
#파라미터 지정
param_grid = {
    'learning_rate': [0.05, 0.01],  
    'n_estimators': [100, 200],
    'max_depth': [5, 10],
}
model = XGBRegressor(random_state=random_state)
tuning_result = my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=-1)

- param_gird 실행 중 발생하는 패키지 오류 경고문을 'pip install -U setuptools' 명령어 실행으로 해결

In [ ]:
model = XGBRegressor(max_depth=10, n_estimators=100, random_state=random_state)

In [ ]:
#예측결과 out put 함수
def save_submission(model, train, y, test, model_name, rmsle=None):
    model.fit(train, y)
    prediction = model.predict(test)
    prediction = np.expm1(prediction)
    data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'
    submission_path = join(data_dir, 'sample_submission.csv')
    submission = pd.read_csv(submission_path)
    submission['price'] = prediction
    submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, model_name, rmsle)
    submission.to_csv(submission_csv_path, index=False)
    print('{} saved!'.format(submission_csv_path))

In [ ]:
#결과 csv로 변환
save_submission(model, train, y, test, 'lgbm', rmsle='0.164399')

## 회고 - 김새봄
### 어려웠던 점 / 느낀점
> - 높은 예측값을 찾기 위한 최적의 파라미터를 찾는 과정이 어려웠음
> - 

### 학습한 내용
> - 예측값을 높이기 위한 파라미터 찾는 방법
> - 결과를 csv로 변환해서 캐글에 업로드하는 방법